Get Resident Advisor files, using @dirkjbreeuwer repo. "resident-advisor-events-scraper"

In [1]:
# ---- IMPORT ------
import os

from jedi.inference.references import gitignored_paths
from win32comext.mapi.mapitags import pidSecureProfileMin

from resident_advisor_events_scraper.event_fetcher import EventFetcher
from datetime import datetime, timedelta

In [7]:
# ---------- OUTPUT ------------
os.makedirs("upcoming_events", exist_ok=True)
output_file = os.path.join(os.getcwd(), "upcoming_events", "ra_events.csv")

# ---------- PARAMETERS ----------
area_code = 13
days_ahead = 7
start_date = datetime.now().strftime('%Y-%m-%d')
end_date = (datetime.now() + timedelta(days=days_ahead)).strftime('%Y-%m-%d')
listing_date_gte = f"{start_date}T00:00:00.000Z"
listing_date_lte = f"{end_date}T23:59:59.999Z"

# ---------- INITIALISE FETCHER ----------
fetcher = EventFetcher(area_code, listing_date_gte, listing_date_lte)

# ---------- FETCH EVENTS ----------
all_events = fetcher.fetch_all_events()

# ---------- SAVE TO CSV ----------
fetcher.save_events_to_csv(all_events, output_file)

print(f"Saved {len(all_events)} events to {output_file}")
print(f"Date of scrape: {start_date}")


Saved 261 events to C:\Users\ADMIN\OneDrive\Documents\Github\Event-Mapper\upcoming_events\ra_events.csv
Date of scrape: 2025-09-02


Upload file to Supabase table

In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from supabase import create_client, Client

load_dotenv()  # This loads the .env file

url = os.getenv('SUPABASE_URL')
key = os.getenv('SUPABASE_ANON_KEY')
supabase: Client = create_client(url, key)

In [8]:
# After your scraping code generates the CSV
df = pd.read_csv('upcoming_events/ra_events.csv')

# Clean the data - replace NaN values
df = df.fillna('')  # Replace NaN with empty strings

# Optional: Convert any remaining problematic values
df = df.replace([np.inf, -np.inf], '')  # Handle infinity values too

# Remove duplicates based on URL column
original_count = len(df)
df = df.drop_duplicates(subset=['Event URL'], keep='first')
duplicates_removed = original_count - len(df)

print(f"Original events: {original_count}")
print(f"Duplicates removed: {duplicates_removed}")
print(f"After removing duplicates: {len(df)} events")

Original events: 261
Duplicates removed: 9
After removing duplicates: 252 events


In [9]:

# Convert to list of dicts for Supabase
events_data = df.to_dict('records')

# Insert with upsert to handle duplicates
try:
    result = supabase.table('resident_advisor').upsert(events_data, on_conflict='Event URL').execute()
    print(f"Uploaded {len(result.data)} events")
except Exception as e:
    print(f"Error uploading: {e}")

Uploaded 252 events
